In [92]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import scipy.io
from tqdm import tqdm
from time import process_time 
import time

## Load data

In [93]:
mat = scipy.io.loadmat('linear_svm.mat')
X_train = mat["X_train"]
X_test = mat["X_test"]
y_train = mat["labels_train"]
y_test = mat["labels_test"]

## Gradient descent optimizer

In [106]:
def gd(iter, X, C, y, w, b, prev_iter=0):
    t = prev_iter * X.shape[0]
    for _ in range(iter):
        for i in range(X.shape[0]):
            t += 1
            eta = 1 / np.sqrt(t)
            if y[i] * (np.dot(w, X[i]) + b) < 1:
                w = w - eta * (w - C * y[i] * X[i])
                b = b + eta * C * y[i]
            else:
                w = w - eta * w

    return w, b

In [ ]:
w = np.zeros([X_train.shape[1]])
b = 0
C = 5
N = X_train.shape[0]

train_acc = []
test_acc = []
stop = False
iter = 0
prev_cost = 1000000
tol = 1e-5
t1 = time.time()
MAX_ITER = 10000
while iter < MAX_ITER:
    w, b = gd(1, X_train, C, y_train, w, b, prev_iter=iter)
    cost = 0.5 * np.dot(w,w) + C * np.maximum(np.zeros([N,]), 1 - y_train * (np.dot(w, X_train.T) + b)).sum()
    cost_improvement = abs(prev_cost - cost) / cost
    prev_cost = cost
    if cost_improvement < tol:
        stop = True
    iter += 1
    train_acc.append((np.sign(np.matmul(X_train, w) + b) == y_train.squeeze()).sum() / y_train.shape[0])
    test_acc.append((np.sign(np.matmul(X_test, w) + b) == y_test.squeeze()).sum() / y_test.shape[0])        

t2 = time.time()
print(f"Solution found in {iter} iterations, test acc {test_acc[-1]}")

## Evaluate model
On test set

In [ ]:
print("************ Accuracy *****************")
(np.sign(np.matmul(X_test, w) + b) == y_test.squeeze()).sum() / y_test.shape[0]

In [ ]:
print(f"Time for execution: {t2-t1}") 

In [98]:
# Save parameters
np.save("w_SDP", w)
np.save("b_SDP", b)